In [1]:
from datasets import load_dataset, load_from_disk
import wordninja
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import re
import glob
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import datasets
import transformers
from transformers import AutoTokenizer, default_data_collator, get_scheduler
from hf_transformers.src.transformers.models.bert.configuration_bert import BertConfig
from hf_transformers.src.transformers.models.bert.modeling_bert import BertForPreTraining

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

import sys
import math

import collections


from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


/usr/local/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
df_hdfs_anomaly = pd.read_csv("./logdata/hdfs/anomaly_label.csv")

In [49]:
def log2list(path):
    with open(path, "r") as f:
        lines = [line.rstrip("\n") for line in f]
    return lines

hdfs = log2list("./logdata/hdfs/HDFS.log")
log_label_pairs = {}

df_hdfs_logs = pd.DataFrame({"log" : hdfs})
df_hdfs_logs["BlockId"] = df_hdfs_logs["log"].map(lambda line : re.search(r"blk_[-]*[0-9]+", line).group(0))
df_hdfs = df_hdfs_anomaly.merge(df_hdfs_logs, how="left", on=["BlockId"])

df_hdfs_without_duplicate = df_hdfs.drop_duplicates(["BlockId"])
df_hdfs_without_duplicate.to_csv("./logdata/hdfs/df_hdfs_without_duplicate.csv")

  0%|          | 553/11175629 [00:37<101:05:47, 30.71it/s]

,BlockId,Label,log
0,blk_-1608999687919862906,Normal,081109 203518 143 INFO dfs.DataNode$DataXceive...
1,blk_-1608999687919862906,Normal,081109 203518 35 INFO dfs.FSNamesystem: BLOCK*...
2,blk_-1608999687919862906,Normal,081109 203519 143 INFO dfs.DataNode$DataXceive...
3,blk_-1608999687919862906,Normal,081109 203519 145 INFO dfs.DataNode$DataXceive...
4,blk_-1608999687919862906,Normal,081109 203519 145 INFO dfs.DataNode$PacketResp...
...,...,...,...
11175624,blk_-9128742458709757181,Anomaly,081111 110423 31 INFO dfs.FSNamesystem: BLOCK*...
11175625,blk_-9128742458709757181,Anomaly,081111 110423 31 INFO dfs.FSNamesystem: BLOCK*...
11175626,blk_-9128742458709757181,Anomaly,081111 110423 33 INFO dfs.FSNamesystem: BLOCK*...
11175627,blk_-9128742458709757181,Anomaly,081111 110423 33 INFO dfs.FSNamesystem: BLOCK*...


In [64]:
# df_hdfs_without_duplicate["text"] = df_hdfs_without_duplicate["log"].map(preprocess_log)

In [66]:
import transformers
transformers.__version__

'4.31.0.dev0'

In [3]:
#分類と事前学習で異なるトークナイザを利用している？
#根拠: 例えばHDFSの異常検知の場合, 異常ラベルはログデータのブロック番号と紐づけられている

In [74]:
#NOTE
#ここで記号を区切ってwordninjaで塊除去を行っている
#なお, かなり時間がかかるので実行には注意が必要
#もう少し賢く書きたかったが, その書き方を思いつく時間とナイーブに実行する時間はどうせ同じぐらいだろう
def preprocess_log(text, remove_digits=True):
    text = text.replace('[', " ")
    text = text.replace(".", " ")
    text = text.replace(",", " ")
    text = text.replace(":", " ")
    text = text.replace("/", " ")
    text = text.replace(";", " ")
    text = text.replace("=", " ")
    text = text.replace("*", " ")
    text = text.replace("_", " ")
    text = text.replace("-", " ")
    text = text.lower()
    text = " ".join(wordninja.split(text))
    remove_num = lambda eg : " ".join([word for word in eg.split(" ") if not word.isdigit()])
    if remove_digits:
        return remove_num(text)
    return text;

def preprocess_log_batch(example, remove_digits=True):
    return {
        "text": [preprocess_log(l, remove_digits=remove_digits) for l in example["log"]]
    }

In [8]:
df_ad_hdfs = pd.read_csv("./logdata/hdfs/hdfs_log_label_pairs.csv", usecols=["log", "BlockId", "Label"])
# df_hdfs_without_duplicate["text"] = df_hdfs_without_duplicate["log"].map(preprocess_log)

In [10]:
datased_ad_hdfs = load_dataset("csv", data_files="./logdata/hdfs/hdfs_log_label_pairs.csv")
datased_ad_hdfs = datased_ad_hdfs["train"].train_test_split(0.2)
datased_ad_hdfs = datased_ad_hdfs.remove_columns(['Unnamed: 0'])
datased_ad_hdfs

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1098.27it/s]


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-81d414b48ce383ec/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00,  9.85it/s]


DatasetDict({
    train: Dataset({
        features: ['log', 'BlockId', 'Label'],
        num_rows: 8940503
    })
    test: Dataset({
        features: ['log', 'BlockId', 'Label'],
        num_rows: 2235126
    })
})

In [69]:
dataset_hdfs = load_dataset("csv", data_files="./logdata/hdfs/df_hdfs_without_duplicate.csv")
dataset_hdfs = dataset_hdfs["train"].train_test_split(0.2)
dataset_hdfs = dataset_hdfs.remove_columns(['Unnamed: 0'])



Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 857.20it/s]

Generating train split: 0 examples [00:00, ? examples/s]
Generating train split: 50000 examples [00:00, 431756.18 examples/s]
Generating train split: 100000 examples [00:00, 414683.88 examples/s]
Generating train split: 160000 examples [00:00, 452425.90 examples/s]
Generating train split: 220000 examples [00:00, 487579.96 examples/s]
Generating train split: 280000 examples [00:00, 501330.62 examples/s]
Generating train split: 340000 examples [00:00, 499879.29 examples/s]
Generating train split: 400000 examples [00:00, 498635.30 examples/s]
Generating train split: 460000 examples [00:00, 507186.77 examples/s]
Generating train split: 520000 examples [00:01, 509134.14 examples/s]
Generating train split: 575061 examples [00:01, 502304.49 examples/s]
                                                                     

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1325ab7ded81eb20/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.



100%|██████████| 1/1 [00:00<00:00, 133.18it/s]


In [70]:
dataset_hdfs

DatasetDict({
    train: Dataset({
        features: ['BlockId', 'Label', 'log'],
        num_rows: 460048
    })
    test: Dataset({
        features: ['BlockId', 'Label', 'log'],
        num_rows: 115013
    })
})

In [72]:
train_data = dataset_hdfs["train"]
test_valid = dataset_hdfs["test"].train_test_split(0.5)
test_dataset = test_valid["test"]
eval_dataset = test_valid["train"]
test_valid

DatasetDict({
    train: Dataset({
        features: ['BlockId', 'Label', 'log'],
        num_rows: 57506
    })
    test: Dataset({
        features: ['BlockId', 'Label', 'log'],
        num_rows: 57507
    })
})

In [76]:
train_data

Dataset({
    features: ['BlockId', 'Label', 'log', 'text'],
    num_rows: 460048
})

In [ ]:
train_data = train_data.map(
    preprocess_log_batch,
    batched=True,
)

In [77]:
eval_dataset = eval_dataset.map(
    preprocess_log_batch,
    batched=True,
)

test_dataset = test_dataset.map(
    preprocess_log_batch,
    batched=True,
)


Map: 100%|██████████| 57507/57507 [00:53<00:00, 1070.86 examples/s]
                                                                   

In [36]:
train_data

Dataset({
    features: ['log', 'BlockId', 'Label'],
    num_rows: 8940503
})

In [78]:
log_tokenizer = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_large/")
log_tokenizer_w_n = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_without_numbers/")

batch_size=16  # change to 16 for full training

def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    batch["labels"] = list(map(lambda x : int(x == "Normal"), batch["Label"]))

    return log_tokenizer_w_n(batch["text"], padding="max_length", truncation=True, max_length=180)



In [ ]:
tokenized_train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=1000, 
    remove_columns=['log', 'BlockId', 'Label', 'text']
)
tokenized_train_data

In [86]:
tokenized_train_data.set_format(type="torch")

In [88]:
small_train_dataset = tokenized_train_data.shuffle(seed=42).select(range(1000))


In [ ]:
eval_dataset = eval_dataset.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=['log', 'BlockId', 'Label', 'text']
)


eval_dataset.set_format(type="torch")
small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(1000))

In [157]:
test_dataset = test_dataset.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=['log', 'BlockId', 'Label', 'text']
)


test_dataset.set_format(type="torch")
test_dataloader = DataLoader(test_dataset, batch_size=8)

In [148]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_train_data, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(eval_dataset, batch_size=8)

In [171]:
from hf_transformers.src.transformers.models.bert.modeling_bert import BertForSequenceClassification
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
unilogConfigmodel = BertForSequenceClassification.from_pretrained("./logdata/unilog_pretrain_preln_on_attentions_0/", num_labels=2).to(device)
optimizer = AdamW(unilogConfigmodel.parameters(), lr=5e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./logdata/unilog_pretrain_preln_on_attentions_0/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [172]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [151]:
import evaluate

In [169]:
del tqdm

In [170]:
from tqdm.auto import tqdm

In [ ]:
train_steps = 0

save_steps = 10000

best_f1 = 0.0

for epoch in range(num_epochs):
    unilogConfigmodel.train()
    for batch in tqdm(train_dataloader):
        train_steps += 1
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = unilogConfigmodel(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        if train_steps % save_steps == 0:
            unilogConfigmodel.save_pretrained(f"./logdata/log_anomaly_model_step_{train_steps}")
    import evaluate

    metric_acc = evaluate.load("accuracy")
    metric_f1 = evaluate.load("f1")
    metric_precision = evaluate.load("precision")
    metric_recall = evaluate.load("recall")

    unilogConfigmodel.eval()
    for batch in tqdm(eval_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = unilogConfigmodel(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric_acc.add_batch(predictions=predictions, references=batch["labels"])
        metric_f1.add_batch(predictions=predictions, references=batch["labels"])
        metric_precision.add_batch(predictions=predictions, references=batch["labels"])
        metric_recall.add_batch(predictions=predictions, references=batch["labels"])

    print(metric_acc.compute())
    print(metric_precision.compute())
    print(metric_recall.compute())
    
    curr_f1_score = metric_f1.compute()["f1"]
    prev_f1_score = best_f1
    if (curr_f1_score > prev_f1_score):
        best_f1 = max(best_f1, curr_f1_score)
        unilogConfigmodel.save_pretrained(f"./logdata/log_anomaly_model_best")


100%|██████████| 7189/7189 [00:47<00:00, 151.82it/s]


{'accuracy': 0.9707508781692346}
{'precision': 0.9707508781692346}
{'recall': 1.0}


  6%|▌         | 3421/57506 [01:05<15:46, 57.15it/s]

In [167]:
import evaluate

metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")
metric_precision = evaluate.load("precision")
metric_recall = evaluate.load("recall")

unilogConfigmodel.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = unilogConfigmodel(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric_acc.add_batch(predictions=predictions, references=batch["labels"])
    metric_f1.add_batch(predictions=predictions, references=batch["labels"])
    metric_precision.add_batch(predictions=predictions, references=batch["labels"])
    metric_recall.add_batch(predictions=predictions, references=batch["labels"])
    
print(metric_acc.compute())
# print(metric_f1.compute())
print(metric_precision.compute())
print(metric_recall.compute())

{'accuracy': 0.9705427165388562}
{'precision': 0.9705427165388562}
{'recall': 1.0}


0.9850511824920578

In [163]:
metric_f1

EvaluationModule(name: "f1", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    labels (`list` of `int`): The set of labels to include when `average` is not set to `'binary'`, and the order of the labels if `average` is `None`. Labels present in the data can be excluded, for example to calculate a multiclass average ignoring a majority negative class. Labels not present in the data will result in 0 components in a macro average. For multilabel targets, labels are column indices. By default, all labels in `predictions` and `references` are used in sorted order. Defaults to None.
    pos_label (`int`): The class to be considered the positive class, in the case where `average` is set to `binary`. Defaults to 1.
    average (`string`): This parameter is required for multiclass/multilabel t